In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Conv1D, Dropout, Flatten, Dense, Conv2D
from matplotlib import pyplot as plt
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [2]:
os.getcwd()

resnet152_input_shape = (224, 224)

inceptionV3_input_shape = (299, 299)

batch_size = 20

In [3]:
path_to_data = "../MIT_Model/test_images"

for f in os.listdir(path_to_data):
    print(f)

Preictal_One
Preictal_Two
Seizure
Interictal


In [4]:
labels = [item for item in os.listdir(path_to_data)]
labels

['Preictal_One', 'Preictal_Two', 'Seizure', 'Interictal']

In [5]:
generator = ImageDataGenerator(
    1./255,
    validation_split=0.2
)

In [6]:
inception_train_gen = generator.flow_from_directory(
    path_to_data,
    batch_size=batch_size, 
    target_size=resnet152_input_shape, 
    classes=list(labels), 
    shuffle=False,
    subset="training")

Found 131 images belonging to 4 classes.


In [7]:
inception_validation_gen = generator.flow_from_directory(
    path_to_data,
    batch_size=batch_size, 
    target_size=resnet152_input_shape, 
    classes=list(labels), 
    shuffle=False,
    subset="validation")

Found 32 images belonging to 4 classes.


In [8]:
def limit_data_samples(data_dir):
  a = []
  for i in os.listdir(data_dir):
    for index, item in enumerate(os.listdir(data_dir+'/'+i)):
      a.append((i))
  return  pd.DataFrame(a, columns=['class'])

In [9]:
test_df2 = limit_data_samples(path_to_data)

In [10]:
classes = test_df2['class'].unique()
classes

array(['Preictal_One', 'Preictal_Two', 'Seizure', 'Interictal'],
      dtype=object)

In [11]:
from sklearn.utils import compute_class_weight

cw = compute_class_weight('balanced', classes, test_df2['class'])

cw = dict(enumerate(cw))

cw

/usr/local/Caskroom/miniforge/base/envs/speciale_01_01/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=['Preictal_One' 'Preictal_Two' 'Seizure' 'Interictal'], y=0      Preictal_One
1      Preictal_One
2      Preictal_One
3      Preictal_One
4      Preictal_One
           ...     
158      Interictal
159      Interictal
160      Interictal
161      Interictal
162      Interictal
Name: class, Length: 163, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


{0: 0.9939024390243902,
 1: 0.9939024390243902,
 2: 1.01875,
 3: 0.9939024390243902}

In [17]:
from tensorflow.keras.applications import VGG16

imported_convnet = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))


imported_convnet

58892288/58889256 [==============================] - 5s 0us/step


In [23]:
model = models.Sequential()
model.add(imported_convnet)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 1028      
Total params: 21,138,500
Trainable params: 21,138,500
Non-trainable params: 0
_________________________________________________________________


In [25]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.01)

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

/usr/local/Caskroom/miniforge/base/envs/speciale_01_01/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [26]:
history = model.fit(
    inception_train_gen,
    epochs=40,
    steps_per_epoch = inception_train_gen.samples//batch_size, 
    validation_data=inception_validation_gen, 
    validation_steps = inception_validation_gen.samples//batch_size,
    
)

/usr/local/Caskroom/miniforge/base/envs/speciale_01_01/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/40


2021-10-26 23:56:08.007181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - ETA: 0s - loss: 1.3867 - accuracy: 0.1261    

2021-10-26 23:56:17.209789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 10s 2s/step - loss: 1.3867 - accuracy: 0.1261 - val_loss: 1.3876 - val_accuracy: 0.2000
Epoch 2/40
6/6 [==============================] - 9s 1s/step - loss: 1.3869 - accuracy: 0.1261 - val_loss: 1.3875 - val_accuracy: 0.0000e+00
Epoch 3/40
6/6 [==============================] - 9s 2s/step - loss: 1.3863 - accuracy: 0.2973 - val_loss: 1.3878 - val_accuracy: 0.0000e+00
Epoch 4/40
6/6 [==============================] - 9s 1s/step - loss: 1.3863 - accuracy: 0.2973 - val_loss: 1.3882 - val_accuracy: 0.0000e+00
Epoch 5/40
6/6 [==============================] - 9s 2s/step - loss: 1.3868 - accuracy: 0.2973 - val_loss: 1.3886 - val_accuracy: 0.0000e+00
Epoch 6/40
6/6 [==============================] - 10s 2s/step - loss: 1.3867 - accuracy: 0.2973 - val_loss: 1.3892 - val_accuracy: 0.0000e+00
Epoch 7/40
6/6 [==============================] - 9s 2s/step - loss: 1.3872 - accuracy: 0.1833 - val_loss: 1.3892 - val_accuracy: 0.0000e+00
Epoch 8/40
6/6 [==========

KeyboardInterrupt: 